# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [10]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [11]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [13]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [15]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [16]:
try:
    session.set_keyspace('project_2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




**Query 1:** In the table **songs_by_id_session**, **session_id** and **item_in_session** columns define the PARTITION KEY to filter the result.

In [43]:
# create table for Query 1
query = "CREATE TABLE IF NOT EXISTS songs_by_id_session"
query = query + """
                (session_id int, 
                item_in_session int, 
                artist text, 
                length float, 
                song text, 
                PRIMARY KEY((session_id, item_in_session))
                )
                """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [44]:
# insert data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO songs_by_id_session(
                session_id, 
                item_in_session, 
                artist, 
                length, 
                song
                )
                """
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        indices = [8, 3, 0, 5, 9]
        types = [int, int, str, float, str]
        session.execute(query, (t(line[i]) for i, t in zip(indices, types)))

In [45]:
## verify the query
query = """SELECT artist, length, song
           FROM songs_by_id_session
           WHERE session_id = 338 AND item_in_session = 4
           LIMIT 10"""

try:
    for row in session.execute(query):
        print(row)
except Exception as e:
    print(e)

Row(artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')


***Query 2:*** In the table **artists_by_uid_session**, **user_id** and **session_id** columns define the PARTITION KEY to filter the result. And **item_in_session** column is used as clustering column to sort the result.

In [46]:
# create table for Query 2
query = "CREATE TABLE IF NOT EXISTS artists_by_uid_session"
query = query + """
                (
                user_id int,
                session_id int, 
                item_in_session int,
                artist text,
                song text,
                first_name text,
                last_name text,
                PRIMARY KEY((user_id, session_id), item_in_session)
                )
                """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [47]:
# insert data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO artists_by_uid_session(
                user_id, 
                session_id, 
                item_in_session, 
                artist,
                song,
                first_name,
                last_name
                )
                """
                
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        indices = [10, 8, 3, 0, 9, 1, 4]
        types = [int, int, int, str, str, str, str]
        session.execute(query, (t(line[i]) for i, t in zip(indices, types)))

In [48]:
## verify the query
query = """
        SELECT artist, song, first_name, last_name
        FROM artists_by_uid_session
        WHERE user_id = 10 AND session_id = 182
        ORDER BY item_in_session DESC
        """

try:
    for row in session.execute(query):
        print(row)
except Exception as e:
    print(e)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')


***Query 3:*** In the table **artists_by_uid_session**, **song** column define the PARTITION KEY to filter the result. And **user_id** column is used as clustering column to sort the result.

In [50]:
# create table for Query 3
query = "CREATE TABLE IF NOT EXISTS users_by_song_uid"
query = query + """
                (
                song text,
                user_id int,
                first_name text,
                last_name text,
                PRIMARY KEY((song), user_id)
                )
                """

try:
    session.execute(query)
except Exception as e:
    print(e)          

In [51]:
# insert data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                INSERT INTO users_by_song_uid(
                song, 
                user_id, 
                first_name, 
                last_name
                )
                """
                
        query = query + "VALUES(%s, %s, %s, %s)"
        indices = [9, 10, 1, 4]
        types = [str, int, str, str]
        session.execute(query, (t(line[i]) for i, t in zip(indices, types)))

In [52]:
## verify the query
query = """
        SELECT first_name, last_name
        FROM users_by_song_uid
        WHERE song = 'All Hands Against His Own'
        """

try:
    for row in session.execute(query):
        print(row)
except Exception as e:
    print(e)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [ ]:
def drop_table(table_names):
    
    '''
    This function helps in drop the tables in the KEYSAPCE.
    arg:
        table_names(list): list of table names to drop in the KEYSPACE
    return:
        None
    '''
    for table in table_names:
        query = "DROP TABLE "+table
        try:
            session.execute(query)
        except Exception as e:
            print(e)
            
# List of table names to drop
table_names = ['songs_by_id_session', 'artists_by_uid_session', 'users_by_song_uid']

# Drop tables
drop_table(table_names)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()